# Stochastic simulations of a novel pathogen

So far in this course, we have only been working with deterministic models, where we get the same output every time we run the model with a given input. This week, you have been introduced to the concept of stochasticity, or random events, in modelling. Random events can be particularly important in determining the course of an epidemic at the beginning of the outbreak, when only a few people are infected. This is what you will be exploring in this exercise.

We are modelling introduction of a novel pathogen into a completely susceptible population. In the cells below, we have provided you with the R code for a simple stochastic SIR model, implemented using the "GillespieSSA" package. This package allows you to quickly set up a Gillespie algorithm, like the one you heard about in the video. While we won't cover how to code a stochastic algorithm in R here, we will provide you with external resources on this at the end of the course if you are interested to learn more. For now, the focus is on understanding the behaviour of stochastic models, and how this compares to that of deterministic models!

## Part 1: Comparing deterministic and stochastic model output when R<sub>0</sub> = 0.75

The following cell contains code for a simple *deterministic* SIR model, which you will be very familiar with. Familiarise yourself with the input: what are the initial conditions, and the parameter values? Then, run the code to simulate and plot the deterministic model output - here, the generated plot shows the prevalence of infection. 

Confirm that the infection goes extinct without causing an epidemic, as you would expect for a determinstic model with R<sub>0</sub> < 1.

In [ ]:
library(deSolve)
library(ggplot2)

# INPUT
initial_state_values <- c(S = 1000000,  
                          I = 1,       
                          R = 0)

parameters <- c(beta = 0.3,
                gamma = 0.4)

times <- seq(from = 0, to = 100, by = 1)

# SIR MODEL FUNCTION
sir_model <- function(time, state, parameters) {  
  
  with(as.list(c(state, parameters)), {
    
    N <- S+I+R
    
    lambda <- beta * I/N
    
    # The differential equations
    dS <- -lambda * S               
    dI <- lambda * S - gamma * I
    dR <- gamma * I             
    
    # Output
    return(list(c(dS, dI, dR))) 
  })
}

output <- as.data.frame(ode(y = initial_state_values, 
                            times = times, 
                            func = sir_model,
                            parms = parameters))

# PLOT OF THE MODEL FIT

ggplot() +
  geom_line(data = output, aes(x = time, y = I)) +                              
  xlab("Time (days)")+                                              
  ylab("Prevalence of infection") +                                 
  labs(title = paste("Deterministic model output for R0 =",parameters["beta"]/parameters["gamma"])) +
  ylim(c(0,150))

Now, run the following code blocks. The first cell performs a single stochastic simulation with the same parameters as the deterministic model, and plots the resulting prevalence of infection. Every time you run the second cell, another iteration of the model will be simulated, and the output added to the plot in a different colour. Run the second cell at least 10 times in a row, then look at the plot of the prevalence of infection.

### Question: What do you observe? How does this compare to the deterministic model output?

In [ ]:
# Simulate stochastic algorithm once:

library("GillespieSSA")

# Defining the model and input
a <- c("beta*S*I/1000000","gamma*I")
nu <- matrix(c(-1,+1,0,0,-1,+1),
             nrow=3,
             ncol=2,
             byrow=FALSE)
tf <- 100

# Simulation
sir_out <- ssa(initial_state_values, a, nu, parameters, tf=tf, simName="SIR")

while(sir_out$stats$nSteps==1){
  sir_out <- ssa(initial_state_values,a,nu, parameters,tf=tf,simName="SIR")
}

# Record number of simulations
n_sims <- 1

# Plot
stoch_plot <- ggplot(as.data.frame(sir_out$data)) +
    geom_line(aes(x = t, y = I)) +
    xlab("Time (days)")+                                              
    ylab("Prevalence of infection") +                                 
    labs(title = paste("Stochastic model output for R0 =",parameters["beta"]/parameters["gamma"]),
         subtitle = paste(n_sims, "simulations")) +
    ylim(c(0,150)) +
    xlim(c(0,100))
  
stoch_plot

In [ ]:
# Add stochastic simulations to the plot:

sir_out <- ssa(initial_state_values,a,nu,parameters,tf=tf,simName="SIR")

while(sir_out$stats$nSteps==1){
  sir_out <- ssa(initial_state_values,a,nu,parameters,tf=tf,simName="SIR")
}

n_sims <- n_sims+1

stoch_plot <- stoch_plot + 
  geom_line(data = as.data.frame(sir_out$data), aes(x = t, y = I), col = sample(rainbow(20),1)) +
    labs(title = paste("Stochastic model output for R0 =",parameters["beta"]/parameters["gamma"]),
         subtitle = paste(n_sims, "simulations"))

stoch_plot

## Part 2: Stochastic simulations with a range of R<sub>0</sub>

Now that you are familiar with what a stochastic model prevalence output looks like for R<sub>0</sub> < 1 in comparison to that from a deterministic model, we will explore the stochastic model behaviour for different values of R<sub>0</sub>. 

Additionally, we are also now interested in the **cumulative incidence of infection** - in each iteration, how many people have become infected in total over our simulation period of 100 days? So far in this etivity (and throughout the entire course), to keep things simple we have only looked at **prevalence** output - the number or proportion of people that are infected at a given timepoint. In many situations, we also want to know what the **incidence** of infection is - how many people have become infected in a given time period. In our model output, the cumulative incidence is just the total number of infections that have occurred by a given point in time. For example, if 1 person becomes infected on day 1 and 2 people become infected on day 2, then the cumulative incidence at day 2 equals 3.

To make the following exercises easier for you, below we have defined a function called *simulate_stoch_model()* that will simulate the stochastic model. In the function call, you can choose between plotting the prevalence of infection or the cumulative incidence, for given parameter values repeatedly, rather than you having to run it manually like in the example above. If you save the function output in a variable, it will also show you the cumulative incidence by 100 days (or by the time the epidemic goes extinct, if this happens before 100 days). Load this function now by running the cell below. We also need to define a new vector of initial conditions, to save the cumulative incidence as an output. Of course, at the beginning of the simulation, the cumulative incidence is 0.

In [ ]:
source("w9_function.R")

initial_state_values <- c(S = 1000000,  
                          I = 1,       
                          R = 0,
                          cum_inc = 0)

The input arguments for the function are $\beta$, $\gamma$, the number of simulations and the output we want to plot. To repeat part 1 of the etivity you would run the following command:

In [ ]:
stoch_output1 <- simulate_stoch_model(beta = 0.3, gamma = 0.4, n_sims = 10, plot = "prevalence")

Note that the limits of the y axis are not pre-defined this time, so they will be automatically chosen based on the highest peak prevalence from any iteration.

Now, let's plot the cumulative incidence instead, as follows:

In [ ]:
stoch_output1 <- simulate_stoch_model(beta = 0.3, gamma = 0.4, n_sims = 10, plot = "cumulative_incidence")

The plot should correspond to the stored cumulative incidence:

In [ ]:
stoch_output1

With R<sub>0</sub> = 0.75, you may observe the occasional epidemic with a cumulative incidence of close to 100, but in most iterations the total number of people infected over the simulation period will not exceed 10-20 people.

Now, keeping $\gamma$ fixed at 0.4 per day, simulate the stochastic model for the following scenarios:  
R<sub>0</sub> = 0.1  
R<sub>0</sub> = 0.9  
R<sub>0</sub> = 1.1    
For each scenario, simulate 100 iterations (it may take a little time for the output to appear).

### Question: For each of the scenarios, how often do you get an epidemic?

In [ ]:
### YOUR CODE GOES HERE ###